# library

In [31]:
# %%capture
# # !pip install -q nnAudio
# !pip install -q --upgrade wandb
# !pip install -q grad-cam
# # !pip install -q ttach
# # !pip install efficientnet_pytorch
# # !pip install albumentations
# !pip install line_profiler
# !pip install transformers
# !pip install audiomentations
# !pip3 install pydub

In [32]:
# !pip install "ipykernel<6"
# !pip install "jupyterlab<3.1"

In [33]:
import os
import gc
import json
import random
from datetime import datetime
import time
import collections
import itertools
from itertools import chain, combinations
import sys
import json
import wandb

import h5py
from glob import glob
import pickle

import scipy as sp
import numpy as np
np.set_printoptions(precision=5, suppress=True) 

import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import IPython.display
from tqdm.auto import tqdm
from skimage.transform import resize

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as torch_functional
from torch.optim import Adam, SGD, AdamW
from torch.optim.lr_scheduler import (CosineAnnealingWarmRestarts,
                    CosineAnnealingLR, ReduceLROnPlateau,_LRScheduler,CyclicLR)
from torch.cuda.amp import autocast, GradScaler

from transformers import get_cosine_schedule_with_warmup

import audiomentations as A
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, PolarityInversion


In [34]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Configuration

In [62]:
class Config:

    #frequently changed 
    model_name = 'TCNN'
    model_version = "main_35th_GeM_vflip_shuffle01_5fold" 
    use_pretrain = False
    use_pseudo_label = False

    debug = False
    use_checkpoint = False
    use_lr_finder = False
    use_subset = False 
    subset_frac = 0.4

    #preproc related
    #augmentation
    vflip = True
    
    time_shift = False
    time_stretch = False

    divide_std = False#std changed... tbs
    shuffle_channels = False #need normalization first
    add_gaussian_noise = False #need normalization first
    shuffle01 = True
    timemask = False
    shift_channel = False    
    pitch_shift = False
    use_mixup = False
    mixup_alpha = 0.1
    cropping = False

    #logistic
    seed = 48
    target_size = 1
    target_col = 'target'
    n_fold = 5
#     gdrive = './drive/MyDrive/Kaggle/G2Net/input/'
    kaggle_json_path = 'kaggle/kaggle.json'
    output_dir = "G2Net-Model/"

    #logger
    print_num_steps=350
    
    #training related
    train_folds = [0,1,2,3,4]
    epochs = 12
    batch_size = 256

    lr= 5e-3#5e-3 # Optimizer 
    weight_decay=0 #1e-4  # Optimizer, default value 0.01
    gradient_accumulation_steps=1 # Optimizer
    scheduler='cosineWithWarmUp' # warm up ratio 0.1 of total steps 
     
    #speedup
    num_workers=0
    non_blocking=True
    amp=True
    use_cudnn = True 
    use_tpu = False
    
    #CNN structure
    channels = 32
    reduction = 1.0

# no need to change below
Config.model_output_folder = Config.output_dir + Config.model_version + "/"
if not os.path.exists(Config.output_dir):
    os.mkdir(Config.output_dir)
if not os.path.exists(Config.model_output_folder):
    os.mkdir(Config.model_output_folder)

torch.backends.cudnn.benchmark = Config.use_cudnn 
display(Config.model_output_folder)

'G2Net-Model/main_35th_GeM_vflip_shuffle01_5fold/'

# wandb

In [36]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

save_object(class2dict(Config), Config.model_output_folder + "Config.pkl")

# Data path

In [37]:
def id_2_path(file_id: str, train=True) -> str:
    if train:
        return "./output/whiten-train/{}.npy".format(file_id)
    else:
        return "./output/whiten-test/{}.npy".format(file_id)

In [38]:
train_df = pd.read_csv('training_labels.csv')
test_df = pd.read_csv('sample_submission.csv')
if Config.debug:
    Config.epochs = 1
    train_df = train_df.sample(n=50000, random_state=Config.seed).reset_index(drop=True)
if Config.use_subset:
    train_df = train_df.sample(frac=Config.subset_frac, random_state=Config.seed).reset_index(drop=True)
train_df['file_path'] = train_df['id'].apply(lambda x :id_2_path(x))
test_df['file_path'] = test_df['id'].apply(lambda x :id_2_path(x,False))

In [39]:
# checking magnitude of waves
num_files = 5
input_file_paths = train_df['file_path'].values[:num_files]
batch_waves=np.zeros((num_files,3,4096))
for i,input_file_path in enumerate(input_file_paths[:num_files]):
    file_name = input_file_path.split('/')[-1].split('.npy')[0]
    waves = np.load(input_file_path)#.astype(np.float32) # (3, 4096)
    # batch_waves[i,:] = np.array([waves.max(),np.abs(waves).max(),np.abs(waves).min()])
    whitened_waves = waves#whiten(waves)
    print(whitened_waves[2][16])

0.014399681
-0.003310739
-0.02684033
0.01655793
-0.02000091


In [40]:
# !!
skf = StratifiedKFold(n_splits=Config.n_fold, shuffle=True, random_state=Config.seed)
splits = skf.split(train_df, train_df["target"])
train_df['fold'] = -1
for fold, (train_index, valid_index) in enumerate(splits):
    train_df.loc[valid_index,"fold"] = fold
train_df['fold_orig'] = train_df['fold']

train_df.groupby('fold')['target'].apply(lambda s: s.value_counts(normalize=True))

fold   
0     0    0.500125
      1    0.499875
1     0    0.500125
      1    0.499875
2     0    0.500125
      1    0.499875
3     0    0.500125
      1    0.499875
4     0    0.500125
      1    0.499875
Name: target, dtype: float64

In [41]:
train_df

,id,target,file_path,fold,fold_orig
0,00000e74ad,1,./output/whiten-train/00000e74ad.npy,3,3
1,00001f4945,0,./output/whiten-train/00001f4945.npy,0,0
2,0000661522,0,./output/whiten-train/0000661522.npy,4,4
3,00007a006a,0,./output/whiten-train/00007a006a.npy,0,0
4,0000a38978,1,./output/whiten-train/0000a38978.npy,4,4
...,...,...,...,...,...
559995,ffff9a5645,1,./output/whiten-train/ffff9a5645.npy,3,3
559996,ffffab0c27,0,./output/whiten-train/ffffab0c27.npy,1,1
559997,ffffcf161a,1,./output/whiten-train/ffffcf161a.npy,2,2
559998,ffffd2c403,0,./output/whiten-train/ffffd2c403.npy,1,1


# Pseudo Labeling

In [42]:
## Decides Threshold based on previous oof pred distribution 

# need to work on the copy, tbs

# if Config.use_pseudo_label:
#     print("Load Checkpoint, epo")
#     checkpoint = torch.load(Config.output_dir + 'SE_reduction1_SiLU/Fold_0_best_model.pth')
#     valid_preds = checkpoint['valid_preds']
#     kf = StratifiedKFold(n_splits=Config.n_fold, shuffle=True, random_state=Config.seed)
#     train_df["fold"] = -1
#     for fold, (train_index, valid_index) in enumerate(kf.split(train_df, train_df["target"])):
#         train_df.loc[valid_index,"fold"] = fold
#     # sanity check
#     def get_score(y_true, y_pred):
#         score = roc_auc_score(y_true, y_pred)
#         return score
#     y_true  = train_df.query("fold == 0")["target"]
#     train_df.drop(columns=["fold"],inplace=True)
#     print(get_score(y_true, valid_preds))
#     # get threshold
#     up_thresh = 0.9
#     down_thresh = 0.1
#     for up_thresh in range(6, 10):
#         up_thresh /= 10
#         for down_thresh in range(4,0,-1):
#             down_thresh /= 10
#             ratio_up = sum(valid_preds > up_thresh) / len(valid_preds)
#             ratio_down = sum(valid_preds < down_thresh) / len(valid_preds)
#             acc_up = np.mean(y_true[ valid_preds > up_thresh])
#             acc_down = 1 - np.mean(y_true[ valid_preds < down_thresh])
#             print("Under Threshold Up : {:.0%}, Down: {:.0%}".format(up_thresh, down_thresh))
#             print("We can have Up sample: {:.1%}, Down sample: {:.1%}".format(ratio_up, ratio_down))
#             print("Up Accuracy: {:.1%}, Down Accuracy: {:.1%}".format(acc_up, acc_down))
#             print()

## Add pseudo label

In [43]:
if Config.use_pseudo_label:
    up_thresh = 0.9
    down_thresh = 0.2

    pseudo_label_df = pd.read_csv(Config.gdrive + "pseudo_label_2.csv")
    pseudo_label_df.head()

    num_test = pseudo_label_df.shape[0]
    num_yes = (pseudo_label_df["target"] >= up_thresh).sum()
    num_no = (pseudo_label_df["target"] <= down_thresh).sum()
    num_all = num_yes+num_no
    print("{:.2%} ratio, {:.2%} 1, {:.2%} 0".format(num_all/num_test, num_yes/num_test, num_no/num_test))
    test_df_2 = pseudo_label_df[(pseudo_label_df["target"] >= up_thresh) | (pseudo_label_df["target"] <= down_thresh)].copy()
    test_df_2["target"] = (test_df_2["target"] > up_thresh).astype(int)
    test_df_2 = test_df_2.merge(test_df[["id","file_path"]],on="id",how="left")
    kf = StratifiedKFold(n_splits=Config.n_fold, shuffle=True, random_state=Config.seed)
    test_df_2['fold'] = -1
    for fold, (train_index, valid_index) in enumerate(kf.split(test_df_2, test_df_2["target"])):
        test_df_2.loc[valid_index,"fold"] = fold
    train_df = pd.concat([train_df, test_df_2]).reset_index(drop=True)
    display(train_df.groupby('fold')['target'].apply(lambda s: s.value_counts(normalize=True)))
    train_df.reset_index(inplace=True, drop=True)
    display(train_df.shape)

train_df

,id,target,file_path,fold,fold_orig
0,00000e74ad,1,./output/whiten-train/00000e74ad.npy,3,3
1,00001f4945,0,./output/whiten-train/00001f4945.npy,0,0
2,0000661522,0,./output/whiten-train/0000661522.npy,4,4
3,00007a006a,0,./output/whiten-train/00007a006a.npy,0,0
4,0000a38978,1,./output/whiten-train/0000a38978.npy,4,4
...,...,...,...,...,...
559995,ffff9a5645,1,./output/whiten-train/ffff9a5645.npy,3,3
559996,ffffab0c27,0,./output/whiten-train/ffffab0c27.npy,1,1
559997,ffffcf161a,1,./output/whiten-train/ffffcf161a.npy,2,2
559998,ffffd2c403,0,./output/whiten-train/ffffd2c403.npy,1,1


# Model

## dataset

In [44]:
transform_list = []

if Config.add_gaussian_noise:
    transform_list.append(A.AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5))
if Config.time_shift:
    transform_list.append(A.Shift(min_fraction=-512*1.0/4096, max_fraction=-1*1.0/4096, p=0.5,rollover=False))#<0 means shift towards left,  fraction of total sound length
# if Config.shift_channel:
#     transform_list.append()
if Config.pitch_shift:
    transform_list.append(A.PitchShift(min_semitones=-1, max_semitones=1, p=0.5))
if Config.time_stretch:
    transform_list.append(A.TimeStretch(min_rate=0.98, max_rate=1.02,leave_length_unchanged=True, p=0.5))
if Config.timemask:
    transform_list.append(A.TimeMask(min_band_part=0.0, max_band_part=0.01, fade=False, p=0.5))#try 0.03 next time
# if Config.vflip:
#     transform_list.append(A.PolarityInversion(p=0.5))

train_transform = A.Compose(transform_list)
# 

test_transform = A.Compose([])
class DataRetriever(Dataset):
    def __init__(self, paths, targets, transforms=None):
        self.paths = paths
        self.targets = targets
        self.transforms = transforms
        # self.ta_augment = ta_Compose([
        #     ta_ShuffleChannels(),
        # ])#bad coding style        
        start_time =time.time()
        array_shape = (len(self.paths),3,4096)
        self.data = np.zeros(array_shape,dtype=np.float32)
        for i,path in enumerate(self.paths):
            waves = np.load(path)
            self.data[i,:] = waves
        print(time.time()-start_time)


    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        
        # path = self.paths[index] 
        # waves = np.load(path)
        if Config.cropping:
            waves = self.data[index][:,1792:3840+1]
        else:
            waves = self.data[index]
        if Config.divide_std:
            waves[0] *= 0.03058
            waves[1] *= 0.03058
            waves[2] *= 0.03096

        if Config.shuffle_channels:
            if np.random.random()<0.5:
                np.random.shuffle(waves)
        if Config.shuffle01:
            if np.random.random()<0.5:
                waves[[0,1]]=waves[[1,0]]
        if Config.vflip:
            if np.random.random()<0.5:
                waves = -waves
              
        if self.transforms is not None:
            waves= self.transforms(waves,sample_rate=2048)
        waves = torch.from_numpy(waves) 
        # if Config.ta:#on tensor, batch*channel*ts
        #     waves = self.ta_augment(waves,sample_rate=2048)
        target = torch.tensor(self.targets[index],dtype=torch.float)#device=device, 
            
        return (waves, target)

class DataRetrieverTest(Dataset):
    def __init__(self, paths, targets, transforms=None):
        self.paths = paths
        self.targets = targets
        self.transforms = transforms

        array_shape = (len(self.paths),3,4096)
        self.data = np.zeros(array_shape,dtype=np.float32)
        for i,path in enumerate(self.paths):
            waves = np.load(path)
            self.data[i,:] = waves  

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        # path = self.paths[index] 
        # waves = np.load(path)
        waves = self.data[index]

        if Config.divide_std:
            waves[0] *= 0.03058
            waves[1] *= 0.03058
            waves[2] *= 0.03096

        if self.transforms is not None:
            waves= self.transforms(waves,sample_rate=2048)
        waves = torch.from_numpy(waves) 
        target = torch.tensor(self.targets[index],dtype=torch.float)#device=device, 
            
        return (waves, target)

class DataRetrieverLRFinder(Dataset):
    def __init__(self, paths, targets, transforms=None):
        self.paths = paths
        self.targets = targets
        self.transforms = transforms
        # self.ta_augment = ta_Compose([
        #     ta_ShuffleChannels(),
        # ])#bad coding style        
#         start_time =time.time()
#         array_shape = (len(self.paths),3,4096)
#         self.data = np.zeros(array_shape,dtype=np.float32)
#         for i,path in enumerate(self.paths):
#             waves = np.load(path)
#             self.data[i,:] = waves
#         print(time.time()-start_time)


    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        
        path = self.paths[index] 
        waves = np.load(path)
        
#         waves = self.data[index]

        if Config.divide_std:
            waves[0] *= 0.03058
            waves[1] *= 0.03058
            waves[2] *= 0.03096

        if Config.shuffle_channels:
            if np.random.random()<0.5:
                np.random.shuffle(waves)
        if Config.shuffle01:
            if np.random.random()<0.5:
                waves[[0,1]]=waves[[1,0]]
        if Config.vflip:
            if np.random.random()<0.5:
                waves = -waves
              
        if self.transforms is not None:
            waves= self.transforms(waves,sample_rate=2048)
        waves = torch.from_numpy(waves) 
        # if Config.ta:#on tensor, batch*channel*ts
        #     waves = self.ta_augment(waves,sample_rate=2048)
        target = torch.tensor(self.targets[index],dtype=torch.float)#device=device, 
            
        return (waves, target)

In [45]:
class GeM(nn.Module):
    '''
    Code modified from the 2d code in
    https://amaarora.github.io/2020/08/30/gempool.html
    '''
    def __init__(self, kernel_size=8, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.kernel_size = kernel_size
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return torch_functional.avg_pool1d(x.clamp(min=eps).pow(p), self.kernel_size).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

## neural net

In [46]:
#for SE-----------------------------------------------------------------------
class SELayer(nn.Module):
    def __init__(self, channel, reduction):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, int(channel // reduction), bias=False),
            nn.SiLU(inplace=True),
            nn.Linear(int(channel // reduction), channel, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class SEBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, reduction,downsample=True):
        super(SEBasicBlock, self).__init__()
        if downsample:
            self.residual_function = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                nn.SiLU(inplace=True),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                SELayer(out_channels, reduction),
                nn.MaxPool1d(2,ceil_mode=True), # downsampling by 2
            )
#             self.shortcut = nn.Sequential(
#                     nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
#                     nn.BatchNorm1d(out_channels),
#                     nn.MaxPool1d(2,ceil_mode=True),  # downsampling by 2
#                 )#skip layers in residual_function, can try simple MaxPool1d
            self.shortcut = nn.Sequential(
                    nn.MaxPool1d(2,ceil_mode=True),  # downsampling by 2
                )
        else:
            self.residual_function = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                nn.SiLU(inplace=True),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                SELayer(out_channels, reduction),
            )
#             self.shortcut = nn.Sequential(
#                     nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
#                     nn.BatchNorm1d(out_channels),
#                 )#skip layers in residual_function, can try identity, i.e., nn.Sequential()
            self.shortcut = nn.Sequential()

    def forward(self, x):
        return nn.SiLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

#-------------------------------------------------------------------------------        
class ResNet(nn.Module):

    def __init__(self, block, num_block):
        super().__init__()

        self.in_channels = Config.channels

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, Config.channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(Config.channels),
            nn.SiLU(inplace=True))

        self.conv2_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[0])
        self.conv3_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[1])
        self.conv4_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[2])
        self.conv5_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[3])
        self.conv6_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[4])
        self.conv7_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[5])
        self.conv8_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[6])
        self.conv9_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[7])
        self.conv10_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[8])
        self.head = nn.Sequential(nn.Flatten(),	#  8*channels
                                  nn.Linear(8*Config.channels, 64),	
                                  nn.SiLU(inplace=True),
                                  nn.Dropout(p=.25),#after activation
                                  nn.Linear(64, 1),	
        )

    def _make_stage(self, block, out_channels, num_blocks):
        """one stage may
        contain more than one residual block
        Args:
            block: block type, basic block, bottle neck block, SE-type block etc
            out_channels: output depth channel number of this layer
            num_blocks: how many blocks for this stage
        Return:
            return a resnet stage
        """
        downsample = [True] + [False] * (num_blocks - 1)
        layers = []
        for i in range(num_blocks):
            layers.append(block(self.in_channels, out_channels,reduction=Config.reduction, downsample=downsample[i]))
            self.in_channels = out_channels 

        return nn.Sequential(*layers)

    def forward(self, x):

        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.conv6_x(output)
        output = self.conv7_x(output)
        output = self.conv8_x(output)
        output = self.conv9_x(output)
        output = self.conv10_x(output)
        output = self.head(output)

        return output



#---------------------------------------------------------------------------------------------------
class StochasticDepthBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, reduction,downsample,p,is_train=True):#tbs for test data
        super(StochasticDepthBasicBlock, self).__init__()
        self.p = p
        self.is_train = is_train
        if downsample:
            self.residual_function = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                nn.SiLU(inplace=True),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                SELayer(out_channels, reduction),
                nn.MaxPool1d(2,ceil_mode=True), # downsampling by 2
            )
            self.shortcut = nn.Sequential(
                    nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
                    nn.BatchNorm1d(out_channels),
                    nn.MaxPool1d(2,ceil_mode=True),  # downsampling by 2
                )#skip layers in residual_function, can try simple MaxPool1d
        else:
            self.residual_function = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                nn.SiLU(inplace=True),
                nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),	
                nn.BatchNorm1d(out_channels),
                SELayer(out_channels, reduction),
            )
            self.shortcut = nn.Sequential(
                    nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
                    nn.BatchNorm1d(out_channels),
                )#skip layers in residual_function, can try identity, i.e., nn.Sequential()
    def survival(self):
        var = torch.bernoulli(torch.tensor(self.p).float())
        return torch.equal(var,torch.tensor(1).float().to(var.device))
    def forward(self, x):
        if self.is_train:
            if self.survival():
                x = nn.SiLU(inplace=True)(self.residual_function(x) + self.shortcut(x))
            else:
                x = self.shortcut(x)
        else:
            x = self.residual_function(x)*self.p+self.shortcut(x)  #what's self.p right now?
            print("p",self.p)
        return x
class StochasticDepthResNet(nn.Module):
    def __init__(self, block, num_block):
        super().__init__()

        self.in_channels = Config.channels

        self.conv1 = nn.Sequential(
            nn.Conv1d(3, Config.channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(Config.channels),
            nn.SiLU(inplace=True))
        
        self.step = (1-0.8)/(sum(num_block)-1)
        self.pl = 1

        self.conv2_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[0])
        self.conv3_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[1])
        self.conv4_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[2])
        self.conv5_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[3])
        self.conv6_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[4])
        self.conv7_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[5])
        self.conv8_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[6])
        self.conv9_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[7])
        self.conv10_x = self._make_stage(block, out_channels=Config.channels, num_blocks=num_block[8])
        self.head = nn.Sequential(nn.Flatten(),	#  8*channels
                                  nn.Linear(8*Config.channels, 64),	
                                  nn.SiLU(inplace=True),
                                  nn.Dropout(p=.25),#after activation
                                  nn.Linear(64, 1),	
        )

    def _make_stage(self, block, out_channels, num_blocks):
        """one stage may
        contain more than one residual block
        Args:
            block: block type, basic block, bottle neck block, SE-type block etc
            out_channels: output depth channel number of this layer
            num_blocks: how many blocks for this stage
        Return:
            return a resnet stage
        """
        downsample = [True] + [False] * (num_blocks - 1)
        layers = []
        for i in range(num_blocks):
            layers.append(block(self.in_channels, out_channels,reduction=Config.reduction,downsample=downsample[i],p=self.pl))
            self.in_channels = out_channels 
            self.pl -= self.step

        return nn.Sequential(*layers)

    def forward(self, x):

        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.conv6_x(output)
        output = self.conv7_x(output)
        output = self.conv8_x(output)
        output = self.conv9_x(output)
        output = self.conv10_x(output)
        output = self.head(output)

        return output

class ModelCNN_Dilations(nn.Module):
    """1D convolutional neural network with dilations. Classifier of the gravitaitonal waves
    Inspired by the https://arxiv.org/pdf/1904.08693.pdf
    """

    def __init__(self):
        super().__init__()
        self.init_conv = nn.Sequential(nn.Conv1d(3, 256, kernel_size=1), nn.ReLU())
        self.convs = nn.ModuleList(
            [
                nn.Sequential(
                    nn.Conv1d(256, 256, kernel_size=2, dilation=2 ** i),
                    nn.ReLU(),
                )
                for i in range(11)
            ]
        )
        self.out_conv = nn.Sequential(nn.Conv1d(256, 1, kernel_size=1), nn.ReLU())
        self.fc = nn.Linear(2049, 1)

    def forward(self, x):
        x = self.init_conv(x)
        for conv in self.convs:
            x = conv(x)
        x = self.out_conv(x)
        x = self.fc(x)
        x.squeeze_(1)
        return x


class Model1DCNN(nn.Module):
    """1D convolutional neural network. Classifier of the gravitational waves.
    Architecture from there https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.120.141103
    """

    def __init__(self, initial_channnels=8):
        super().__init__()
        self.cnn1 = nn.Sequential(
            nn.Conv1d(3, initial_channnels, kernel_size=64),
            nn.BatchNorm1d(initial_channnels),
            nn.ELU(),
        )
        self.cnn2 = nn.Sequential(
            nn.Conv1d(initial_channnels, initial_channnels, kernel_size=32),
            nn.MaxPool1d(kernel_size=8),
            nn.BatchNorm1d(initial_channnels),
            nn.ELU(),
        )
        self.cnn3 = nn.Sequential(
            nn.Conv1d(initial_channnels, initial_channnels * 2, kernel_size=32),
            nn.BatchNorm1d(initial_channnels * 2),
            nn.ELU(),
        )
        self.cnn4 = nn.Sequential(
            nn.Conv1d(initial_channnels * 2, initial_channnels * 2, kernel_size=16),
            nn.MaxPool1d(kernel_size=6),
            nn.BatchNorm1d(initial_channnels * 2),
            nn.ELU(),
        )
        self.cnn5 = nn.Sequential(
            nn.Conv1d(initial_channnels * 2, initial_channnels * 4, kernel_size=16),
            nn.BatchNorm1d(initial_channnels * 4),
            nn.ELU(),
        )
        self.cnn6 = nn.Sequential(
            nn.Conv1d(initial_channnels * 4, initial_channnels * 4, kernel_size=16),
            nn.MaxPool1d(kernel_size=4),
            nn.BatchNorm1d(initial_channnels * 4),
            nn.ELU(),
        )
        if Config.cropping:
            fm_size = tbd
        else:
            fm_size = 11
        self.fc1 = nn.Sequential(
            nn.Linear(initial_channnels * 4 * fm_size, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ELU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ELU(),
        )
        self.fc3 = nn.Sequential(
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = self.cnn4(x)
        x = self.cnn5(x)
        x = self.cnn6(x)
        # print(x.shape)
        x = x.flatten(1)
        # x = x.mean(-1)
        # x = torch.cat([x.mean(-1), x.max(-1)[0]])
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

    
class Model1DCNNGEM(nn.Module):
    """1D convolutional neural network. Classifier of the gravitational waves.
    Architecture from there https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.120.141103
    """

    def __init__(self, initial_channnels=8):
        super().__init__()
        self.cnn1 = nn.Sequential(
            nn.Conv1d(3, initial_channnels, kernel_size=64),
            nn.BatchNorm1d(initial_channnels),
            nn.ELU(),
        )
        self.cnn2 = nn.Sequential(
            nn.Conv1d(initial_channnels, initial_channnels, kernel_size=32),
            GeM(kernel_size=8),
            nn.BatchNorm1d(initial_channnels),
            nn.ELU(),
        )
        self.cnn3 = nn.Sequential(
            nn.Conv1d(initial_channnels, initial_channnels * 2, kernel_size=32),
            nn.BatchNorm1d(initial_channnels * 2),
            nn.ELU(),
        )
        self.cnn4 = nn.Sequential(
            nn.Conv1d(initial_channnels * 2, initial_channnels * 2, kernel_size=16),
            GeM(kernel_size=6),
            nn.BatchNorm1d(initial_channnels * 2),
            nn.ELU(),
        )
        self.cnn5 = nn.Sequential(
            nn.Conv1d(initial_channnels * 2, initial_channnels * 4, kernel_size=16),
            nn.BatchNorm1d(initial_channnels * 4),
            nn.ELU(),
        )
        self.cnn6 = nn.Sequential(
            nn.Conv1d(initial_channnels * 4, initial_channnels * 4, kernel_size=16),
            GeM(kernel_size=4),
            nn.BatchNorm1d(initial_channnels * 4),
            nn.ELU(),
        )
        if Config.cropping:
            fm_size = tbd
        else:
            fm_size = 11
        self.fc1 = nn.Sequential(
            nn.Linear(initial_channnels * 4 * fm_size, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ELU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ELU(),
        )
        self.fc3 = nn.Sequential(
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = self.cnn4(x)
        x = self.cnn5(x)
        x = self.cnn6(x)
        # print(x.shape)
        x = x.flatten(1)
        # x = x.mean(-1)
        # x = torch.cat([x.mean(-1), x.max(-1)[0]])
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x    


In [47]:
# def Model():
#     """ return a skip connection network
#     """
#     return ResNet(SEBasicBlock, [1,1,1,1,1,1,1,1,1])
# def Model():
#     """ return a StochasticDepthResNet network
#     """
#     return StochasticDepthResNet(StochasticDepthBasicBlock, [1,2,1,1,1,1,1,1,1])
# def Model():
#     return Model1DCNN(Config.channels)
def Model():
    return Model1DCNNGEM(Config.channels)

In [48]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
model = Model()#can possibly call random
get_n_params(model)

659204

## Util

In [49]:
def get_score(y_true, y_pred):
    score = roc_auc_score(y_true, y_pred)
    return score

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch(seed=Config.seed)    

def get_scheduler(optimizer, train_size):
    if Config.scheduler=='ReduceLROnPlateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=Config.factor, 
                                      patience=Config.patience, verbose=True, eps=Config.eps)
    elif Config.scheduler=='CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, 
                                      T_max=Config.T_max, 
                                      eta_min=Config.min_lr, last_epoch=-1)
    elif Config.scheduler=='CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, 
                                                T_0=Config.T_0, 
                                                T_mult=1, 
                                                eta_min=Config.min_lr, 
                                                last_epoch=-1)
    elif Config.scheduler=='CyclicLR':
        iter_per_ep = train_size/Config.batch_size
        step_size_up = int(iter_per_ep*Config.step_up_epochs)
        step_size_down=int(iter_per_ep*Config.step_down_epochs)
        scheduler = CyclicLR(optimizer, 
                             base_lr=Config.base_lr, 
                             max_lr=Config.max_lr,
                             step_size_up=step_size_up,
                             step_size_down=step_size_down,
                             mode=Config.mode,
                             gamma=Config.cycle_decay**(1/(step_size_up+step_size_down)),
                             cycle_momentum=False)
        
    elif Config.scheduler == 'cosineWithWarmUp':
        epoch_step = train_size/Config.batch_size
        num_warmup_steps = int(0.1 * epoch_step * Config.epochs)
        num_training_steps = int(epoch_step * Config.epochs)
        scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=num_warmup_steps, 
                                                    num_training_steps=num_training_steps)      
    return scheduler
def mixed_criterion(loss_fn, pred, y_a, y_b, lam):
    return lam * loss_fn(pred, y_a) + (1 - lam) * loss_fn(pred, y_b)
def mixup_data(x, y, alpha=1.0):
    """Returns mixed inputs, pairs of targets, and lambda"""
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size, requires_grad=False).to(x.device,non_blocking=Config.non_blocking)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [50]:
# setting device on GPU if available, else CPU
if Config.use_tpu:
    device = xm.xla_device()
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')#for debug, tb see
print('Using device:', device)
print()

#Additional Info when using cuda
# watch nvidia-smi
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Reserved:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA A100-PCIE-40GB
Memory Usage:
Allocated: 0.0 GB
Reserved:    8.5 GB


## LR Finder

In [51]:
class LRFinder:
    def __init__(self, model, optimizer, criterion, device):
        self.optimizer = optimizer
        self.model = model
        self.criterion = criterion
        self.device = device
        torch.save(model.state_dict(), f'{Config.model_output_folder}/init_params.pt')

    def range_test(self, loader, end_lr = 10, num_iter = 100, 
                   smooth_f = 0.05, diverge_th = 5):
        lrs = []
        losses = []
        best_loss = float('inf')
        lr_scheduler = ExponentialLR(self.optimizer, end_lr, num_iter)
        for step, batch in enumerate(loader):
            if step == num_iter:
                break
            loss = self._train_batch(batch)
            lrs.append(lr_scheduler.get_last_lr()[0])
            #update lr
            lr_scheduler.step()
            if step > 0:
                loss = smooth_f * loss + (1 - smooth_f) * losses[-1]
            if loss < best_loss:
                best_loss = loss
            losses.append(loss)
            if loss > diverge_th * best_loss:
                print("Stopping early, the loss has diverged")
                break
        #reset model to initial parameters
        model.load_state_dict(torch.load(f'{Config.model_output_folder}/init_params.pt'))
        return lrs, losses

    def _train_batch(self, batch):
        self.model.train()
        self.optimizer.zero_grad()
        scaler = GradScaler()
        X = batch[0].to(self.device,non_blocking=Config.non_blocking)
        targets = batch[1].to(self.device,non_blocking=Config.non_blocking)
        
        if Config.use_mixup:
            (X_mix, targets_a, targets_b, lam) = mixup_data(
                X, targets, Config.mixup_alpha
            )
            with autocast():
                outputs = self.model(X_mix).squeeze()
                loss = mixed_criterion(self.criterion, outputs, targets_a, targets_b, lam)
        else:
            with autocast():
                outputs = self.model(X).squeeze()
                loss = self.criterion(outputs, targets)
        #loss.backward()
        scaler.scale(loss).backward()
        
        if Config.use_tpu:
            xm.optimizer_step(self.optimizer, barrier=True)  # Note: TPU-specific code! 
        else:
            scaler.step(self.optimizer)
            scaler.update()
#             self.optimizer.step()
        return loss.item()

          
            
                    
        



                    
                    
class ExponentialLR(_LRScheduler):
    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(ExponentialLR, self).__init__(optimizer, last_epoch)
    def get_lr(self):
        curr_iter = self.last_epoch
        r = curr_iter / self.num_iter
        return [base_lr * (self.end_lr / base_lr) ** r for base_lr in self.base_lrs]

def plot_lr_finder(lrs, losses, skip_start = 0, skip_end = 0):
    if skip_end == 0:
        lrs = lrs[skip_start:]
        losses = losses[skip_start:]
    else:
        lrs = lrs[skip_start:-skip_end]
        losses = losses[skip_start:-skip_end]
    fig = plt.figure(figsize = (16,8))
    ax = fig.add_subplot(1,1,1)
    ax.plot(lrs, losses)
    ax.set_xscale('log')
    ax.set_xlabel('Learning rate')
    ax.set_ylabel('Loss')
    ax.grid(True, 'both', 'x')
    plt.show()

In [52]:
if Config.use_lr_finder:
    START_LR = 1e-7
    model = Model()
    model.to(device,non_blocking=Config.non_blocking)
    optimizer = AdamW(model.parameters(), lr=START_LR, weight_decay=Config.weight_decay, amsgrad=False)
    criterion = torch_functional.binary_cross_entropy_with_logits

    train_data_retriever = DataRetrieverLRFinder(train_df['file_path'], train_df["target"].values)
    train_loader = DataLoader(train_data_retriever,
                                batch_size=Config.batch_size, 
                                shuffle=True, 
                                num_workers=Config.num_workers, pin_memory=True, drop_last=True)

In [53]:
%%time
if Config.use_lr_finder:
    try:
        END_LR = 10
        NUM_ITER = 200
        lr_finder = LRFinder(model, optimizer, criterion, device)
        lrs, losses = lr_finder.range_test(train_loader, END_LR, NUM_ITER)
    except RuntimeError as e:
        del model, optimizer, criterion, train_data_retriever, train_loader, lr_finder
        gc.collect()
        torch.cuda.empty_cache() 
        print(e)

CPU times: user 4 µs, sys: 3 µs, total: 7 µs
Wall time: 11.4 µs


In [54]:
if Config.use_lr_finder:
    plot_lr_finder(lrs[:-28], losses[:-28])

## Trainer

In [55]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion, 
        scheduler,
        valid_labels,
        best_valid_score,
        fold,
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.scheduler = scheduler
        self.best_valid_score = best_valid_score
        self.valid_labels = valid_labels
        self.fold = fold

    
    def fit(self, epochs, train_loader, valid_loader, save_path): 
        train_losses = []
        valid_losses = []
        for n_epoch in range(epochs):
            start_time = time.time()
            print('Epoch: ', n_epoch)
            train_loss, train_preds = self.train_epoch(train_loader)
            valid_loss, valid_preds = self.valid_epoch(valid_loader)

            train_losses.append(train_loss)
            valid_losses.append(valid_loss)

            if isinstance(self.scheduler, ReduceLROnPlateau):
                self.scheduler.step(valid_loss)
            valid_score = get_score(self.valid_labels, valid_preds)

            numbers = valid_score
            filename = Config.model_output_folder+f'score_epoch_{n_epoch}.json'          
            with open(filename, 'w') as file_object: 
                json.dump(numbers, file_object) 
            

            if self.best_valid_score < valid_score:
                self.best_valid_score = valid_score
                self.save_model(n_epoch, save_path+f'best_model.pth', train_preds, valid_preds)

            print('train_loss: ',train_loss)
            print('valid_loss: ',valid_loss)
            print('valid_score: ',valid_score)
            print('best_valid_score: ',self.best_valid_score)
            print('time used: ', time.time()-start_time)

            wandb.log({f"[fold{self.fold}] epoch": n_epoch+1, 
                      f"[fold{self.fold}] avg_train_loss": train_loss, 
                      f"[fold{self.fold}] avg_val_loss": valid_loss,
                      f"[fold{self.fold}] val_score": valid_score})        

        # fig,ax = plt.subplots(1,1,figsize=(15,7))
        # ax.plot(list(range(epochs)), train_losses, label="train_loss")
        # ax.plot(list(range(epochs)), valid_losses, label="val_loss")
        # fig.legend()
        # plt.show()            
            
    def train_epoch(self, train_loader):
        if Config.amp:
            scaler = GradScaler()
        self.model.train()
        losses = []
        train_loss = 0
        # preds = []
        for step, batch in enumerate(train_loader, 1):
            self.optimizer.zero_grad()
            X = batch[0].to(self.device,non_blocking=Config.non_blocking)
            targets = batch[1].to(self.device,non_blocking=Config.non_blocking)
            
            if Config.use_mixup:
                (X_mix, targets_a, targets_b, lam) = mixup_data(
                    X, targets, Config.mixup_alpha
                )
                with autocast():
                    outputs = self.model(X_mix).squeeze()
                    loss = mixed_criterion(self.criterion, outputs, targets_a, targets_b, lam)
            else:
                with autocast():
                    outputs = self.model(X).squeeze()
                    loss = self.criterion(outputs, targets)



    
    

                
            if Config.gradient_accumulation_steps > 1:
                loss = loss / Config.gradient_accumulation_steps
            scaler.scale(loss).backward()
          
            if (step) % Config.gradient_accumulation_steps == 0:
                scaler.step(self.optimizer)
                scaler.update()
            

            if (not isinstance(self.scheduler, ReduceLROnPlateau)):
                self.scheduler.step()

            # preds.append(outputs.sigmoid().to('cpu').detach().numpy())
            loss2 = loss.detach()

            wandb.log({f"[fold{self.fold}] loss": loss2,
                       f"[fold{self.fold}] lr": self.scheduler.get_last_lr()[0]})            

            # losses.append(loss2.item())
            losses.append(loss2)
            train_loss += loss2

            if (step) % Config.print_num_steps == 0:
                train_loss = train_loss.item() #synch once per print_num_steps instead of once per batch
                print(f'[{step}/{len(train_loader)}] ', 
                      f'avg loss: ',train_loss/step,
                      f'inst loss: ', loss2.item())
                
        # predictions = np.concatenate(preds)

#         losses_avg = []
#         for i, loss in enumerate(losses):
#             if i == 0 :
#                 losses_avg.append(loss)
#             else:
#                 losses_avg.append(losses_avg[-1] * 0.6 + loss * 0.4)
#         losses = torch.stack(losses)
#         losses_avg = torch.stack(losses_avg)
#         fig,ax = plt.subplots(1,1,figsize=(15,7))
#         ax.plot(list(range(step)), losses, label="train_loss per step")
#         ax.plot(list(range(step)), losses_avg, label="train_loss_avg per step")
#         fig.legend()
#         plt.show()            
        
        return train_loss / step, None#, predictions

    def valid_epoch(self, valid_loader):
        self.model.eval()      
        valid_loss = []
        preds = []
        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                X = batch[0].to(self.device,non_blocking=Config.non_blocking)
                targets = batch[1].to(self.device,non_blocking=Config.non_blocking)
                outputs = self.model(X).squeeze()
                loss = self.criterion(outputs, targets)
                if Config.gradient_accumulation_steps > 1:
                    loss = loss / Config.gradient_accumulation_steps
                valid_loss.append(loss.detach().item())
                preds.append(outputs.sigmoid().to('cpu').numpy())
#                 valid_loss.append(loss.detach())#.item())
#                 preds.append(outputs.sigmoid())#.to('cpu').numpy())
#         valid_loss = torch.cat(valid_loss).to('cpu').numpy()
#         predictions = torch.cat(preds).to('cpu').numpy()
        predictions = np.concatenate(preds)
        return np.mean(valid_loss), predictions

    def save_model(self, n_epoch, save_path, train_preds, valid_preds):
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
                'scheduler': self.scheduler.state_dict(),
                'train_preds': train_preds,
                'valid_preds': valid_preds,
            },
            save_path,
        )
    


# Training loop

In [56]:
seed_torch(seed=Config.seed)

In [57]:
def training_loop(use_checkpoint=Config.use_checkpoint):
    kf = StratifiedKFold(n_splits=Config.n_fold, shuffle=True, random_state=Config.seed)
    avg_best_valid_score = 0
    folds_val_score = []
    for fold in range(Config.n_fold): 
        train_index, valid_index = train_df.query(f"fold!={fold}").index, train_df.query(f"fold_orig=={fold}").index 
        print('Fold: ', fold)
        if fold not in Config.train_folds:
            print("skip")
            continue
        train_X, valid_X = train_df.loc[train_index], train_df.loc[valid_index]
        valid_labels = train_df.loc[valid_index,Config.target_col].values
#         fold_indices = pd.read_csv(f'{Config.gdrive}/Fold_{fold}_indices.csv')#saved fold ids
        oof = pd.DataFrame()
        oof['id'] = train_df.loc[valid_index,'id']
        oof['id'] = valid_X['id'].values.copy()
        oof = oof.reset_index()
        # assert oof['id'].eq(fold_indices['id']).all()
#         if not Config.use_subset:
#             assert oof['id'].eq(fold_indices['id']).sum()==112000
        oof['target'] = valid_labels
        
        oof.to_csv(f'{Config.model_output_folder}/Fold_{fold}_oof_pred.csv')
        # continue # uncomment this is to check oof ids

        print('training data samples, val data samples: ', len(train_X) ,len(valid_X))
        train_data_retriever = DataRetriever(train_X["file_path"].values, train_X["target"].values, transforms=train_transform)#how to run this only once and use for next experiment?
        valid_data_retriever = DataRetrieverTest(valid_X["file_path"].values, valid_X["target"].values, transforms=test_transform)        
        train_loader = DataLoader(train_data_retriever,
                                  batch_size=Config.batch_size, 
                                  shuffle=True, 
                                  num_workers=Config.num_workers, pin_memory=True, drop_last=False)
        valid_loader = DataLoader(valid_data_retriever, 
                                  batch_size=Config.batch_size * 2, 
                                  shuffle=False, 
                                  num_workers=Config.num_workers, pin_memory=True, drop_last=False)

        model = Model()
        model.to(device,non_blocking=Config.non_blocking)
        optimizer = AdamW(model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay, amsgrad=False)
        scheduler = get_scheduler(optimizer, len(train_X))
        best_valid_score = -np.inf
        if use_checkpoint:
            print("Load Checkpoint, epo")
            checkpoint = torch.load(f'{Config.model_output_folder}/Fold_{fold}_best_model.pth')
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            best_valid_score = float(checkpoint['best_valid_score'])
            scheduler.load_state_dict(checkpoint['scheduler'])
        
        
        criterion = torch_functional.binary_cross_entropy_with_logits
        

        trainer = Trainer(
            model, 
            device, 
            optimizer, 
            criterion,
            scheduler,
            valid_labels,
            best_valid_score,
            fold
        )

        history = trainer.fit(
            epochs=Config.epochs, 
            train_loader=train_loader, 
            valid_loader=valid_loader,
            save_path=f'{Config.model_output_folder}/Fold_{fold}_',
        )
        folds_val_score.append(trainer.best_valid_score)
    wandb.finish()
    print('folds score:', folds_val_score)
    print("Avg: {:.5f}".format(np.mean(folds_val_score)))
    print("Std: {:.5f}".format(np.std(folds_val_score)))

# Weight & Bias Initialization

In [58]:
wandb.login(key="1b0833b15e81d54fad9cfbbe3d923f57562a6f89")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/.netrc


True

In [59]:
job_type= "debug" if Config.debug else "train"
# run = wandb.init(project="G2Net", name=Config.model_version, config=class2dict(Config), group=Config.model_name, job_type=job_type)
run = wandb.init(project="G2Net", name=Config.model_version, config=class2dict(Config), group=Config.model_name, job_type=Config.model_version)

# Train

In [60]:
%%time
try:
    %lprun -f DataRetriever.__getitem__ -f Trainer.train_epoch -f Trainer.fit -f Trainer.valid_epoch training_loop() 
#     training_loop(Config.use_checkpoint)
except RuntimeError as e:
    gc.collect()
    torch.cuda.empty_cache()   
    print(e)# saving oof predictions

Fold:  0
skip
Fold:  1
training data samples, val data samples:  448000 112000
336.94975185394287
Epoch:  0
[350/1750]  avg loss:  0.5555198015485491 inst loss:  0.4833698868751526
[700/1750]  avg loss:  0.5092899431501116 inst loss:  0.4461674690246582
[1050/1750]  avg loss:  0.4889393833705357 inst loss:  0.45250049233436584
[1400/1750]  avg loss:  0.4778629847935268 inst loss:  0.45013427734375
[1750/1750]  avg loss:  0.470455810546875 inst loss:  0.4560246765613556
train_loss:  0.470455810546875
valid_loss:  0.5664388945657913
valid_score:  0.8592448548826505
best_valid_score:  0.8592448548826505
time used:  86.73019313812256
Epoch:  1
[350/1750]  avg loss:  0.4342332240513393 inst loss:  0.48556503653526306
[700/1750]  avg loss:  0.4346610804966518 inst loss:  0.45815128087997437
[1050/1750]  avg loss:  0.43389721098400297 inst loss:  0.3697444200515747
[1400/1750]  avg loss:  0.4320674351283482 inst loss:  0.4148033857345581
[1750/1750]  avg loss:  0.43075669642857145 inst loss: 

[fold1] avg_train_loss,0.39132
[fold1] avg_val_loss,0.40484
[fold1] epoch,12
[fold1] loss,0.38439
[fold1] lr,0.0
[fold1] val_score,0.87599
[fold2] avg_train_loss,0.39108
[fold2] avg_val_loss,0.40517
[fold2] epoch,12
[fold2] loss,0.38258
[fold2] lr,0.0


[fold1] avg_train_loss,█▄▄▃▃▃▂▂▂▁▁▁
[fold1] avg_val_loss,▁▁█▁▁▁▁▁▁▁▁▁
[fold1] epoch,▁▂▂▃▄▄▅▅▆▇▇█
[fold1] loss,█▆▆▄▅█▄▅▄▄▅▆▄▄▃▄▄▃▁▄▄▄▆▂▅▆▃▃▄▄▃▅▄▃▂▁▃▁▃▁
[fold1] lr,▂▄▅▇██████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
[fold1] val_score,▇▇▁█████████
[fold2] avg_train_loss,█▄▄▃▃▃▂▂▂▁▁▁
[fold2] avg_val_loss,█▂▄▄▁▄▂▁▁▁▁▁
[fold2] epoch,▁▂▂▃▄▄▅▅▆▇▇█
[fold2] loss,█▆▄▆▃▅▆▄▃▄▄▄▄▂▅▃▃▂▁█▃▂▅▄▃▄▄▄▂▁▁▇▅▂▄▂▂▂▂▁
[fold2] lr,▂▄▅▇██████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁


folds score: [0.8760714600478335, 0.8760509249700195, 0.8779104317826671, 0.8762285171367314]
Avg: 0.87657
Std: 0.00078
CPU times: user 3h 1min 3s, sys: 6h 30min 49s, total: 9h 31min 52s
Wall time: 1h 52min 1s


Timer unit: 1e-06 s

Total time: 1099.92 s
File: <ipython-input-44-065401859317>
Function: __getitem__ at line 42

Line #      Hits         Time  Per Hit   % Time  Line Contents
    42                                               def __getitem__(self, index):
    43                                                   
    44                                                   # path = self.paths[index] 
    45                                                   # waves = np.load(path)
    46  21504000   21233771.0      1.0      1.9          if Config.cropping:
    47                                                       waves = self.data[index][:,1792:3840+1]
    48                                                   else:
    49  21504000   28698601.0      1.3      2.6              waves = self.data[index]
    50  21504000   18701391.0      0.9      1.7          if Config.divide_std:
    51                                                       waves[0] *= 0.03058
    52                      

# Pause

In [ ]:
from jarviscloud import jarviscloud
jarviscloud.pause()

In [63]:
for fold in Config.train_folds:
    print(fold)
    checkpoint = torch.load(f'{Config.model_output_folder}/Fold_{fold}_best_model.pth')
    # print(checkpoint['valid_preds'])
    try:
        # oof = pd.read_csv(f'{Config.gdrive}/Fold_{fold}_indices.csv') also works, used in replacement of next statement for previously not generated Fold_{fold}_oof_pred.csv
        oof = pd.read_csv(f'{Config.model_output_folder}/Fold_{fold}_oof_pred.csv')
        oof['pred'] = checkpoint['valid_preds']
        oof.to_csv(f'{Config.model_output_folder}/Fold_{fold}_oof_pred.csv') 
        print('successfully saved oof predictions for Fold: ', fold)   
    except:
        raise RuntimeError('failure in saving predictions for Fold: ', fold)

0
successfully saved oof predictions for Fold:  0
1
successfully saved oof predictions for Fold:  1
2
successfully saved oof predictions for Fold:  2
3
successfully saved oof predictions for Fold:  3
4
successfully saved oof predictions for Fold:  4


# add TTA

In [ ]:
# dataset

In [64]:
class TTA(Dataset):
    def __init__(self, paths, targets, use_vflip=False, shuffle_channels=False, time_shift=False, add_gaussian_noise = False,  time_stretch=False,shuffle01=False ):
        self.paths = paths
        self.targets = targets
        self.use_vflip = use_vflip
        self.shuffle_channels = shuffle_channels
        self.time_shift = time_shift
        self.gaussian_noise = add_gaussian_noise
        self.time_stretch = time_stretch
        self.shuffle01 = shuffle01
        if time_shift:
            self.time_shift = A.Shift(min_fraction=-512*1.0/4096, max_fraction=-1.0/4096, p=1,rollover=False)
        if add_gaussian_noise:
            self.gaussian_noise = A.AddGaussianNoise(min_amplitude=0.001, max_amplitude= 0.015, p=1)
        if time_stretch:
            self.time_stretch = A.TimeStretch(min_rate=0.9, max_rate=1.111,leave_length_unchanged=True, p=1)
              
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        path = self.paths[index] 
        waves = np.load(path)

        if Config.divide_std:
            waves[0] *= 0.03058
            waves[1] *= 0.03058
            waves[2] *= 0.03096

        if self.use_vflip:
            waves = -waves
        if self.shuffle_channels:
            np.random.shuffle(waves)
        if self.time_shift:
            waves = self.time_shift(waves, sample_rate=2048)
        if self.gaussian_noise:
            waves = self.gaussian_noise(waves, sample_rate=2048)
        if self.time_stretch:
            waves = self.time_stretch(waves, sample_rate=2048)
        if self.shuffle01:
            waves[[0,1]] = waves[[1,0]]
        
        waves = torch.from_numpy(waves) 
        target = torch.tensor(self.targets[index],dtype=torch.float)#device=device,             
        return (waves, target)

In [65]:
## functions for making predictions

In [66]:
def get_pred(loader,model):
    preds = []
    for step, batch in enumerate(loader, 1):
        if step % Config.print_num_steps == 0:
            print("step {}/{}".format(step, len(loader)))
        with torch.no_grad():
            X = batch[0].to(device,non_blocking=Config.non_blocking)
            outputs = model(X).squeeze()
            preds.append(outputs.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

def get_tta_pred(df,model,**transforms):
    data_retriever = TTA(df['file_path'].values, df['target'].values, **transforms)
    loader = DataLoader(data_retriever, 
                            batch_size=Config.batch_size * 2, 
                            shuffle=False, 
                            num_workers=Config.num_workers, pin_memory=True, drop_last=False)
    return get_pred(loader,model)

     

In [67]:
##TTA for oof

In [68]:
model = Model()
oof_all = pd.DataFrame()
for fold in Config.train_folds:
    oof = train_df.query(f"fold=={fold}").copy()
    oof['preds'] = torch.load(f'{Config.model_output_folder}/Fold_{fold}_best_model.pth')['valid_preds']
    oof['file_path'] = train_df['id'].apply(lambda x :id_2_path(x))
    # display(oof)    

    checkpoint = torch.load(f'{Config.model_output_folder}/Fold_{fold}_best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device=device,non_blocking=Config.non_blocking)
    model.eval()


    oof["tta_vflip"] = get_tta_pred(oof,model,use_vflip=True)
    # oof["tta_shift"] = get_tta_pred(oof,model,time_shift=True)
    # oof["tta_vflip_shift"] = get_tta_pred(oof,model,use_vflip=True,time_shift=True)
    oof["tta_shuffle01"] = get_tta_pred(oof,model,shuffle01=True)
    oof["tta_vflip_shuffle01"] = get_tta_pred(oof,model,use_vflip=True,shuffle01=True)
    # oof["tta_shift_shuffle01"] = get_tta_pred(oof,model,time_shift=True,shuffle01=True)
    # oof["tta_vflip_shift_shuffle01"] = get_tta_pred(oof,model,use_vflip=True,time_shift=True,shuffle01=True)

    oof.to_csv(Config.model_output_folder + f"/oof_Fold_{fold}.csv", index=False)
    oof_all = pd.concat([oof_all,oof])




In [69]:
print("Original:",roc_auc_score(oof_all['target'], oof_all['preds']))

for col in oof.columns:
    if "tta" in col:
        print(col,roc_auc_score(oof_all['target'], oof_all[col]))

oof_all['avg']=0
count = 0
for col in oof_all.columns:
    if "tta" in col or 'preds' in col: 
        count+=1
        oof_all['avg'] += oof_all[col]
oof_all['avg'] /= count
print("preds_tta_avg:",roc_auc_score(oof_all['target'], oof_all['avg']))

oof_all.to_csv(Config.model_output_folder + "/oof_all.csv", index=False)
oof_all[['id','fold','avg']].rename(columns={'id':'id','fold':'fold','avg':'prediction'}).to_csv(Config.model_output_folder + "/oof_final.csv", index=False)

Original: 0.8766354666392064
tta_vflip 0.876561745269803
tta_shuffle01 0.8764093953304342
tta_vflip_shuffle01 0.8766249669510503
preds_tta_avg: 0.8774725437897382


In [ ]:
## TTA for test

In [ ]:
%%time 
test_df['target'] = 0  
model = Model()
test_avg = test_df[['id', 'target']].copy()
for fold in Config.train_folds:
    test_df2 = test_df.copy()
    checkpoint = torch.load(f'{Config.model_output_folder}/Fold_{fold}_best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device=device,non_blocking=Config.non_blocking)
    model.eval()

    test_df2['preds'+f'_Fold_{fold}'] = get_tta_pred(test_df2,model)
    test_df2["tta_vflip"+f'_Fold_{fold}'] = get_tta_pred(test_df2,model,use_vflip=True)
#     test_df2["tta_shift"+f'_Fold_{fold}'] = get_tta_pred(test_df2,model,time_shift=True)
#     test_df2["tta_vflip_shift"+f'_Fold_{fold}'] = get_tta_pred(test_df2,model,use_vflip=True,time_shift=True)
    test_df2["tta_shuffle01"+f'_Fold_{fold}'] = get_tta_pred(test_df2,model,shuffle01=True)
    test_df2["tta_vflip_shuffle01"+f'_Fold_{fold}'] = get_tta_pred(test_df2,model,use_vflip=True,shuffle01=True)
#     test_df2["tta_shift_shuffle01"+f'_Fold_{fold}'] = get_tta_pred(test_df2,model,time_shift=True,shuffle01=True)
#     test_df2["tta_vflip_shift_shuffle01"+f'_Fold_{fold}'] = get_tta_pred(test_df2,model,use_vflip=True,time_shift=True,shuffle01=True)
    
    test_df2.to_csv(Config.model_output_folder + f"/test_Fold_{fold}.csv", index=False)
    count = 0
    for col in test_df2.columns:
        if "tta" in col or 'preds' in col: 
            count+=1
            test_avg['target'] += test_df2[col]/len(Config.train_folds)
    test_avg['target'] /= count
test_avg.to_csv(Config.model_output_folder + "/test_avg.csv", index=False)

#just used vflip here

step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442
step 350/442


In [ ]:
# Create Submission File

In [ ]:
test_avg[['id', 'target']].to_csv("./submission.csv", index=False)

test_avg[['id', 'target']].to_csv(Config.model_output_folder + "/submission.csv", index=False)

!mkdir -p ~/.kaggle/ && cp $Config.kaggle_json_path ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle competitions submit -c g2net-gravitational-wave-detection -f ./submission.csv -m $Config.model_version

In [ ]:
# test_avg